In [ ]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
import string
  
 

data = pd.read_csv("../../Data/SMSSpamCollection.txt", sep='\t', header=None)
data.columns = ['label', 'Content']

en_stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

In [ ]:
data

<h3>Now we are goining to crete a variable witch contains the length of the mail content</h3>

In [ ]:
data['Content_len'] = data['Content'].apply(lambda x: len(x) - x.count(" "))

In [ ]:
data.head()

<h3>Now we are goining to crete a function witch return the punctuation rate of email's content</h3>


In [ ]:
def punctuation_rate(text):
    nb_punctuation = sum([1 for ch in text if ch in string.punctuation])
    text_size = len(text) - text.count(" ") # la longueur d'email sans les espaces
    return round(nb_punctuation/(text_size),4)*100
    

In [ ]:
data['punctuation_rate'] = data['Content'].apply(lambda x:punctuation_rate(x) )

In [ ]:
data.head()

<h3>Now we are goining to crete a function witch return the cleaning Content i.e without stopwords and ...</h3>


In [ ]:
def clean_email(text):
    result  = "".join([word for word in text if word not in string.punctuation ])
    splited = re.split('W+',result)
    final = [ps.stem(word) for word in splited if word not in en_stopwords ]
    return final


In [ ]:
vectorization_full = CountVectorizer(analyzer=clean_email)
vect_final = vectorization_full.fit_transform(data['Content'])

In [ ]:
all_data = pd.concat([pd.DataFrame(vect_final.toarray()), data['Content_len'], data['punctuation_rate']],axis=1)

In [ ]:
all_data

<h3>Fiting the model SVM</h3>


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_data,data['label'],test_size=0.2)

In [ ]:
from sklearn import svm

In [ ]:
alg_svm = svm.SVC(kernel='linear')

In [ ]:
alg_svm.fit(X_train,y_train)